In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
spacex_df["Launch Site"].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [4]:
# Create a dash application
app = dash.Dash()

In [5]:
# Create an app layout
app.layout = html.Div(children = [html.H1("SpaceX Launch Records Dashboard",
                                         style = {"textAlign": "center", 
                                                 "color": "#503D36",
                                                 "font-size": 40}),
                                  # TASK 1: Add a dropdown list to enable Launch Site selection
                                  # The default select value is for ALL sites
                                  # dcc.Dropdown(id='site-dropdown',...)
                                  dcc.Dropdown(id = "site-dropdown",
                                              options = [{"label": "All Sites", "value": "All"},
                                                        {"label": "CCAFS LC-40", "value": "CCAFS LC-40"},
                                                        {"label": "VAFB SLC-4E", "value": "VAFB SLC-4E"},
                                                        {"label": "KSC LC-39A", "value": "KSC LC-39A"},
                                                        {"label": "CCAFS SLC-40", "value": "CCAFS SLC-40"}],
                                              value = "All",
                                              placeholder = "Select a Launch Site here",
                                              searchable = True),
                                  html.Br(),
                                  
                                  # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                  # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                  html.Div(dcc.Graph(id='success-pie-chart')),
                                  
                                  html.P("Payload range (Kg):"),
                                  # TASK 3: Add a slider to select payload range
                                  #dcc.RangeSlider(id='payload-slider',...)
                                  dcc.RangeSlider(id = "payload-slider", 
                                                  min = min_payload, 
                                                  max = max_payload, 
                                                  step = 1000, 
                                                  value = [min_payload, max_payload]),
                                  
                                  
                                  # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                  html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])

In [6]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def create_pie_chart(value):
    if value == "All":
        df = spacex_df
    else:
        df = spacex_df[spacex_df["Launch Site"] == value]
        
    df = df.groupby("class").size().reset_index(name = "class count")
        
    fig = px.pie(df, values = "class count", names = "class", title = "Total success launched for selected sites")
    return fig

In [7]:
value = "All"
if value == "All":
    df = spacex_df
else:
    df = spacex_df[spacex_df["Launch Site"] == value]

df = df.groupby("class").size().reset_index(name = "class count")
# df = df.groupby("Launch Site").mean().reset_index()

fig = px.pie(df, values = "class count", names = "class", title = "Total success launched for selected sites")
# fig = px.pie(df, values = "class", names = "Launch Site", title = "Total success launched for selected sites")
# fig.show()

In [8]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def create_scatter_plot(site, payload):
    mini, maxi = payload
    payload_range = (spacex_df["Payload Mass (kg)"] >= mini) & (spacex_df["Payload Mass (kg)"] <= maxi)
    df = spacex_df[payload_range]
    
    if value == "All":
        df = df
    else:
        df = df[spacex_df["Launch Site"] == value]
            
    fig = px.scatter(df, x = "Payload Mass (kg)", y = "class", color = "Booster Version Category")
    return fig

In [9]:
mini, maxi = 0, 10000
payload_range = (spacex_df["Payload Mass (kg)"] >= mini) & (spacex_df["Payload Mass (kg)"] <= maxi)
df = spacex_df[payload_range]

if value == "All":
    df = df
else:
    df = df[spacex_df["Launch Site"] == value]

fig = px.scatter(df, x = "Payload Mass (kg)", y = "class", color = "Booster Version Category")
# fig.show()

In [ ]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [09/Jul/2023 14:10:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 14:10:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 14:10:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 14:10:05] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2023 14:10:05] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2023 14:10:05] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2023 14:10:05] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2023 14:10:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 14:10:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 14:10:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 14:10:07] "POST /_dash-upd